In [1]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self,X):
        return X - X.mean()

In [2]:
layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [3]:
net = nn.Sequential(nn.Linear(8,128),CenteredLayer())

In [4]:
Y = net(torch.rand(4,8))
Y.mean()

tensor(3.7253e-09, grad_fn=<MeanBackward0>)

In [6]:
class MyLinear(nn.Module):
    def __init__(self,in_units,units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units,units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self,X):
        linear = torch.matmul(X,self.weight.data) + self.bias.data
        return F.relu(linear)

In [7]:
linear = MyLinear(5,3)
linear.weight

Parameter containing:
tensor([[ 1.2215,  0.2846,  0.1182],
        [ 0.4349,  0.2714, -0.1410],
        [ 0.1927,  1.2082,  0.4386],
        [ 2.2846, -0.0492,  0.0565],
        [-0.9805,  0.9194,  0.1017]], requires_grad=True)

In [9]:
linear(torch.rand(2,5))

tensor([[0.3284, 0.9948, 0.9174],
        [2.5085, 1.4843, 1.2787]])

In [10]:
net = nn.Sequential(MyLinear(64,8),MyLinear(8,1))
net(torch.rand(2,64))

tensor([[3.0577],
        [0.0515]])

## 作业

In [40]:
class TensorReduction(nn.Module):
    def __init__(self, dim1, dim2):
        super(TensorReduction, self).__init__()
        # 定义一个可训练的权重参数，维度为(dim2, dim1, dim1)
        self.weight = nn.Parameter(torch.rand(dim2, dim1, dim1))

    def forward(self, X):
        # 初始化一个全零张量，大小为(X.shape[0], self.weight.shape[0])
        Y = torch.zeros(X.shape[0], self.weight.shape[0])
        for k in range(self.weight.shape[0]):
            # 计算temp = X @ weight[k] @ X^T
            temp = torch.matmul(X, self.weight[k]) @ X.T
            # 取temp的对角线元素，存入Y[:, k]
            Y[:, k] = temp.diagonal()
        return Y

# 创建一个TensorReduction层，dim1=10, dim2=5
layer = TensorReduction(10, 5)
# 创建一个大小为(2, 10)的张量X
X = torch.rand(2, 10)
# 对layer(X)进行前向传播，返回一个大小为(2, 5)的张量
# layer(X)
layer.weight.shape[0]



5

In [49]:
import torch.nn as nn
import torch.fft as fft

class FourierLayer(nn.Module):
    def __init__(self):
        super(FourierLayer,self).__init__()

    def forward(self,x):
        x = fft.fftn(x)
        x = x[:,:,:x.shape[2] // 2]
        return x

In [50]:
X = torch.rand(1,2,5)
net = FourierLayer()
net(X)

tensor([[[4.1717+0.0000j, 0.8397-0.5480j],
         [1.8629+0.0000j, 0.4215-0.1478j]]])

tensor([[[ 5.1854+0.0000j,  0.2507+0.3938j, -0.0421-0.1534j, -0.0421+0.1534j,
           0.2507-0.3938j],
         [-0.6578+0.0000j,  0.3475-0.4206j,  0.3030+1.4332j,  0.3030-1.4332j,
           0.3475+0.4206j]]])